In [3]:
import torch
import sklearn

In [13]:
print("dsasdd")
print("dsasdd")

dsasdd
dsasdd


In [ ]:
print("32")

In [ ]:
print("356243234242343423423443232423423423423u48253678345632656253jgfsdfgjgsiufg6")

In [ ]:
print("dsa6565sdd")

In [ ]:
print("dsa6sdd")

In [ ]:
print("dsasdd")